# Image Stitching Template
___
### Version 0.2.1
Duncan Muir

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from htbam_analysis.stitching import scripts, rastering, stitching_settings, background_images
import jax
jax.default_device = jax.devices('cpu')[0]

# 1. Stitching
___

## Stitching Setting

In [ ]:
# Settings for stitching
settings = stitching_settings.StitchingSettings(
    setup_num = 1,
    tile_dim = 1600
    )

## Button Quant

In [ ]:
# Path to folder containing background images
button_bkgd_folder = '/path/to/background/folder'

# Path to folder containing button images
button_folder = '/path/to/button/folder'



# Parameters for button quant stitching
bg_button_param = rastering.raster_params.RasterParams(
    overlap=0.1, # 10% overlap
    size=1600, # 800x800 pixel tiles
    acqui_ori=(True, False),
    rotation = -0.6, # valid as of 2023-12-01
    auto_ff = True,
    ff_type = 'BaSiC'
    )

In [ ]:
# run background stitching
scripts.walk_and_stitch(path=button_bkgd_folder,
    stitch_settings=settings, 
    params=bg_button_param,
    stitch_type = 'single')

# run button stitching
scripts.walk_and_stitch(path=button_folder,
    stitch_settings=settings, 
    params=bg_button_param,
    stitch_type = 'single')

# Kinetics

In [ ]:
kin_std_param = rastering.raster_params.RasterParams(
    overlap=0.1, # 10% overlap
    size=1600, # 800x800 pixel tiles
    acqui_ori=(True, False),
    rotation = -0.6
    )

In [ ]:
# Path to folder containing kinetics background images
kinetics_bkgd_folder = '/path/to/background/folder'
# Path to folder containing kinetics images
kinetics_path = '/path/to/kinetics/folder'

In [ ]:
# run kinetics background stitching
scripts.walk_and_stitch(path=kinetics_bkgd_folder,
    stitch_settings=settings, 
    params=kin_std_param, 
    stitch_type = 'kinetic')

# run kinetics stitching
scripts.walk_and_stitch(path=kinetics_path,
    stitch_settings=settings, 
    params=kin_std_param, 
    stitch_type = 'kinetic')

## Standard Curve 

In [ ]:
standard_path = '/Path/to/standard/folder'

# NOTE: Make Sure to Follow This Exact Format!
# ('1_5625{conc_unit}_{FP_name}', '1_5625')
# Both values are strings, replace {conc_unit} with the unit of concentration,
# and replace {FP_name} with the name of the FP.
# i.e.,
# handles = [('3_90625uM_NADPH', "3_90625"), 
#            ('7_8125uM_NADPH', "7_8125"), 
#            ('15_625uM_NADPH', "15_625"), 
#            ('31_25uM_NADPH', "31_25"), 
#            ('62_5uM_NADPH', "62_5"),
#            ('125uM_NADPH', "125"),
#            ('250uM_NADPH', "250"),
#            ('500uM_NADPH', "500")]


handles = [('3_90625uM_NADPH', "3_90625"), 
           ('7_8125uM_NADPH', "7_8125"), 
           ('15_625uM_NADPH', "15_625"), 
           ('31_25uM_NADPH', "31_25"), 
           ('62_5uM_NADPH', "62_5"),
           ('125uM_NADPH', "125"),
           ('250uM_NADPH', "250"),
           ('500uM_NADPH', "500")]

In [ ]:
# run standard stitching
scripts.stitch_standard(standard_path,
                        kin_std_param, 
                        handles)

# 2. Background Subtraction
___

In [10]:
bg = background_images.BackgroundImages()

In [ ]:
device = "1"

button_quant_channel = "2"
button_quant_exposure = 1000

kinetic_channel = "4"
kinetic_exposure = 10

#### Define paths and add to the backgrounds

In [ ]:
# Reference Background Images:

# button_quant_background
button_quant_background_img = "/path/to/stiched/button/quant/background/image"
bg.add(button_quant_background_img, 
       device, 
       button_quant_channel, 
       button_quant_exposure
       )

# kinetics_background
kinetics_background_img = "/path/to/stiched/kinetics/background/image"
bg.add(kinetics_background_img, 
       device, 
       kinetic_channel, 
       kinetic_exposure
       )

#### Define target and execute

In [18]:
button_quant_target = '/path/to/stiched/button/quant/target/image/folder'

bg.walk_and_bg_subtract(button_quant_target, 
                        device, 
                        button_quant_channel)

In [ ]:
kinetic_target = '/path/to/stiched/kinetics/target/image/folder'

bg.walk_and_bg_subtract(kinetic_target,
                        device, 
                        kinetic_channel)

In [ ]:
standard_target = '/path/to/stiched/standard/target/image/folder'

bg.walk_and_bg_subtract(standard_target,
                        device, 
                        kinetic_channel)